In [6]:
%load_ext autoreload
%autoreload 2
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
sys.path.append('/home/iaamini/Documents/ML_practice/ML_fundamentals/ML_fundamentals/algorithms/')

In [9]:
import numpy as np
import logistic_regression as lg
import optimizers as opt

In [50]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=10)

In [11]:
w = np.random.random(X.shape[1]+1)

In [12]:
np.dot(X, w[:X.shape[1]]).sum()

-0.720344839145916

# La régression logistique

- Méthode introduite par les statisticiens vers la fin des années 60 et popularisée par Andersen (1982).
- Permet de s'affranchir des hypothèses restrictives associées aux méthode linéaires paramétriques.
- Hypothèse:
    - Logarithme des rapports de probabilités conditionnelles des classes pour une entrée $x$ est linéaire par rapport à $x$.

$$\ln{\Big(\frac{\mathbb{P}(X =x | Y = 1)}{\mathbb{P}(X = x | Y = -1)}\Big)} = w_0 + \langle \bar{w}, x \rangle$$


- La probabilité à posteriori:

$$\mathbb{P}(Y = 1 | X = x) = \frac{1}{1 + e^{-(\tilde{w} + \langle \bar{w}, x \rangle)}}$$

Ci-dessous se trouve la fonction permettant de calculer l'équation ci-dessus.

In [13]:
# # Le x correspond à formule linéaire de x
# def logistic(x):
#     return (1.0 / (1.0 + np.exp(-x)))

# Lien avec le principe MRE
## Gradient de la fonction de coût

### Fonction de coût:

$$ \hat{\mathcal{L}}(\textbf{w}) = \frac{1}{m}\sum_{t=1}^{m}\ln{(1 + e^{-yh_w(x_i)})}$$



In [14]:
# def logistic_surrogate_loss(X, y, w):
#     """ Calcul de la fonction de coût logistique
#     Paramètres
#     -----------
#     X: matrix, or sparse array shape (n, d)
#     y: array, shape (n,)
#         True labels
#     w: array, shape (d+1,)
#         Weight vectors (the +1 is for the intercept)
#     Renvoie
#     -------
#     loss : float,
#         Valeur de la fonction de coût
#     """
#     n, d = X.shape
#     S = 0.
#     ps = 0.
#     ps += np.dot(X,w[:d]).sum()
#     S += (logistic(y*ps)).sum() / n
#     return S
    

In [15]:
# Test rapide pour vérifier que logistic_surrogate_loss renvoie toujours
# des valeurs entre 0 et 1
# w0 = 2
# w = np.random.random(d)
# logistic_surrogate_loss(X, y, w)

### Le gradient de la fonction:

$$ \nabla \hat{\mathcal{L}}(\textbf{w}) = \frac{1}{m}\sum_{t=1}^{m}y_i \Big(1 - \frac{1}{1 + e^{-y_ih_w(x_i)}}\Big) \times x_i $$

- Pour l'apprentissage des paramètres du modèle de la régression logistique en utilisant la méthode du gradient conjugué  pour minimser 

In [16]:
# def gradient_logistic_surrogate_loss(X, y, w):
#     """Calcul du vecteur gradient Eq. (3.17) avec le biais en plus
#     Paramètres
#     -----------
#     X: matrix, or sparse array shape (n, d)
#     y: array, shape (n,)
#         True labels
#     w: array, shape (d+1,)
#         Weight vectors
#     w0: scalar,
#         bais
#     Renvoie
#     -------
#     grad : array, shape (d+1,)
#         Vecteur gradient de la fonction de coût logistique
#     """
#     n, d = X.shape
#     S = 0.
#     g = np.zeros(d + 1)
#     ps = 0.
#     ps += np.dot(X,w[:d]).sum()
#     g[-1] = 0.
#     for i in range(n):
#         g[-1] += ((logistic(y*ps) - 1.0) * y).sum()
#         g[:d] += np.dot((logistic(y*ps) - 1.0) * y, X)
    
#     g /= n
#     return g

In [17]:
w

array([0.30797592, 0.30269473, 0.4801337 , 0.4119392 , 0.08254573,
       0.55105515, 0.51323093, 0.03623768, 0.063779  , 0.65499226,
       0.18500994])

In [20]:
w0 = 2
# w = np.random.random(d)
lg.gradient_logistic_surrogate_loss(X, y, w)

array([ -2.47377881,   0.12529948,  23.0674621 , -34.90829428,
         3.87118441,  -3.7833038 ,  33.01638623,   5.99995905,
        -0.47292404,   9.11790205, -32.96146448])

In [21]:
w0 = 2
# w = np.random.random(d)
lg.gradient_logistic_surrogate_loss(X, y, w)

array([ -2.47377881,   0.12529948,  23.0674621 , -34.90829428,
         3.87118441,  -3.7833038 ,  33.01638623,   5.99995905,
        -0.47292404,   9.11790205, -32.96146448])

In [22]:
w = np.random.random(X.shape[1]+1)
Lold = lg.logistic_surrogate_loss(X, y, w)

In [23]:
n, d = X.shape

# Apprentissage des paramètres du modèle

La régression linéaire utilise la méthode du gradient conjugué afin de minimiser la fonction de coût.

Le gradient conjugué utilise lui-même une recherche linéaire afin de trouver le pas d'apprentissage optimal.
Un brève présentation de l'algorithme de recherche linéaire s'impose donc.

## recherche linéaire

On cherche une valeur maximale admissible du pas d'apprentissage en allant selon un direction de descente $p_t$ (vérifiant $p_t^T\nabla\mathcal{L}(w)$.
On suit toujours un règle de mise à jour avec la condition de décroissance de la fonction de coût.
$$\forall{t} \in \mathbb{N}, \hat{\mathcal{L}}(w^{(t+1)}) < \hat{\mathcal{L}}(w^{(t)}) $$

L'algorithme de recherche linéaire consiste à vérifier des conditions que l'on appelle *conditions de Wolfe*.

### Condition d'Armijo

Permet de répondre aux deux situations où l'équation ci-dessus peut être satisfaite sans pour autant atteindre le minimiseur de $\mathcal{L}$.

$$\forall{t} \in \mathbb{N}, \hat{\mathcal{L}}(w^{(t)} + \eta_tp_t) \leq \hat{\mathcal{L}}(w_t) + \alpha\eta_tp_t^{T}\nabla\hat{\mathcal{L}}(w_t)$$

la contrainte de décroissance linéaire implique que le taux de décroissance allant $\hat{\mathcal{L}}(w^t)$ à $\hat{\mathcal{L}}(w^{t+1})$ ne doit pas être plus grand que la descente pondéré par un coefficient alpha.

### Condition de courbure

Cette condition implique que la descente lors de l'itération suivante soit au moins égale à une fraction $\beta \in (\alpha, 1)$. Donc:
$$ \forall{t} \in \mathbb{N}, p^T_t\nabla\hat{\mathcal{L}}(w^{(t)} + \eta_tp_t) \geq \beta p^T_t\nabla\hat{\mathcal{L}}(w^{(t)})$$

In [5]:
q = np.arange(10)
p = q + 1

In [30]:
p[1] = 100

In [31]:
q

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [27]:
g = gradient_logistic_surrogate_loss(X, y, w)

In [28]:
p = -g

In [107]:
eta = 0.2
eta2 = 1
a = np.array([[1/eta**2, 1/eta2**2], [-eta2/eta, eta/eta2]])
z = np.array([1-2-6, 3-4-1]).reshape(2, 1)

In [108]:
z

array([[-7],
       [-2]])

In [113]:
(a @ z).reshape(1, -1)[0][0]

-176.99999999999997

In [116]:
ab = 1/(eta-eta2) * np.dot(a,z)

In [119]:
ab = ab.reshape(1, -1)

In [120]:
a, b = ab[0][0], ab[0][1]

In [55]:
a.shape

(1,)

In [148]:
# #TODO:  Harmonize var names
# def line_search(X, y, w, cost_func, p, g, old_loss, new_loss, w_old):
#     """Line search algorithm
#     Parameters
#     ----------
    
#     X : array_like, shape (n, d),
#         Matrice contenant les données
#     y : array, shape (n,)
#         Vecteur des labels
#     w : array, (d+1,)
#         Vecteur des poids avec le biais en plus
#     cost_func : callable,
#         Fonction de coût
#     p : array, (d+1,)
#         Vecteur de la direction de descente
#     g : array (d+1,)
#         Vecteur gradient
#     """
    
#     alpha = 1e-4
#     mineta = 1e-7
    
#     n, d = X.shape
#     # initialisation de w_old
# #     w_old = w.copy()
    
#     # Initialisation de old_loss et new_loss
# #     old_loss = cost_func(X, y, w)
#     new_loss = old_loss
    
#     # Calcul de la pente au point actuel (float)
#     pente = p @ g
    
#     # Définition de la valeur minimale tolérée de eta
#     _max = 0.
#     for j in range(d):
#         if np.abs(p[j]) > _max * np.maximum(np.abs(w_old[j]), 1.):
# #             print(np.abs(p[j]))
#             _max = np.abs(p[j]) / np.maximum(np.abs(w_old[j]), 1.)
# #             print(_max)
#     etamin = mineta / _max
    
#     # Initialisation de eta à 1
#     eta = 1.
#     w = w_old + eta * p
    
#     new_loss = cost_func(X, y, w)
    
#     # Boucler tant que la condition d'Armijo n'est pas complète (Eq. 2.18)
#     while new_loss > (old_loss + alpha * eta * pente):
#         if eta < etamin:
#             w = w_old
#             break
#         else:
#             if eta == 1.:
#                 # Minimiseur du polynôme d'interpolation de degré 2 (Eq. 2.32)
#                 etatmp = -pente / (2*(new_loss - old_loss - pente))
#             else:
#                 coeff1 = new_loss - old_loss - eta*pente
#                 coeff2 = new_loss2 - old_loss - eta2*pente
                
#                 d = np.array([[1/(eta**2), -1/(eta2**2)], [-eta2/(eta**2), eta/(eta2**2)]])
#                 c = np.array([coeff1, coeff2]).reshape(2, 1)
# #                 print(d)
# #                 print(c)
                
#                 ab = (1/(eta-eta2) * np.dot(d,c)).reshape(1, -1)
#                 a, b = ab[0][0], ab[0][1]
                
                
#                 # Calcul des coefficients du polynôme d'interpolation de degré 3 (Eq. 2.33)
#                 #TODO: Use numpy to compute
                
# #                 a = (coeff1/(eta**2) - coeff2/(eta**2)) / (eta - eta2)
# #                 b = (-eta2*coeff1/ eta**2 + eta*coeff2 / (eta2**2)) / (eta - eta2)
#                 if a != 0.:
#                     delta = (b**2)-3.*a*pente
# #                     print(delta)
#                     if delta >= 0.:
#                         # minimiseur du polynôme d'interpolation de degré 3 (Eq 2.34)
#                         etatmp = (-b + np.sqrt(delta)) / (3.*a)
#                     else:
# #                         print("problème d'interpolation")
#                         raise ValueError("rchln:problème d'interpolation")
                        
#                 else:
#                     etatmp = -pente / (2. * b)
#                     if etatmp > 0.5 * eta:
#                         etatmp = 0.5 * eta
#         eta2 = eta
#         new_loss2 = new_loss
#         eta = np.maximum(etatmp, 0.1*eta)
#         w = w_old + eta*p
#         new_loss = cost_func(X, y, w)
#     return w, new_loss

In [104]:
p.shape

(11,)

In [33]:
w1 = np.random.random(d+1)
w2 = np.random.random(d+1)

In [34]:
lg.logistic_surrogate_loss(X, y, w1)

0.2629939632353619

In [35]:
lg.logistic_surrogate_loss(X, y, w2)

0.2550829789266817

In [30]:
w1, test_new_loss = opt.line_search(X, y, w1, cost_func=lg.logistic_surrogate_loss, p=p, g=g)
# w2, test_new_loss = line_search(X, y, w2, cost_func=logistic_surrogate_loss, p=p, g=g)

NameError: name 'p' is not defined

In [147]:
test_new_loss

0.6470570105885094

In [90]:
logistic_surrogate_loss(X, y, w1)

0.255

In [91]:
logistic_surrogate_loss(X, y, w2)

0.255

In [23]:
a = np.arange(11)
b = np.arange(11)

In [24]:
np.maximum(2,0)

2

In [42]:
print(ps.T.shape, g.shape)

(100,) (11,)


In [44]:
# ps @ g

In [46]:
g[-1]

-0.0033798716608000046

In [47]:
g.shape[0]

11

# Gradient conjugué

In [149]:
# def conjugate_gradient(X, y, w, cost_func, grd_func, epsilon):
    
#     epoque = 0
#     n, d = X.shape
#     w_old = np.empty_like(w)
#     p = np.empty_like(w)
    
    
#     new_loss = cost_func(X, y, w)
#     old_loss = new_loss + 2*epsilon
#     g = grd_func(X, y, w)
#     # Initialisation de p au gradient en temps 0 (Eq. 2.46)
#     p = -g
#     print(p)
    
#     while np.abs(old_loss - new_loss) > np.abs(old_loss)*epsilon:
#         old_loss = new_loss
#         w, new_loss =line_search(X=X, y=y, w=w, cost_func=cost_func, 
#                                  p=p, g=g, old_loss=old_loss, new_loss=new_loss, w_old=w_old)
# #         print(w)
#         # Calcul du nouveau vecteur gradient (Eq. 2.42)
#         h = grd_func(X, y, w)
#         dgg = g @ g
#         ngg = h @ h
        
#         # Faut-il rajouter la formule de Ribière-Polak (Eq. 2.52)
        
#         # Formule de Fletcher-Reeves (Eq. 2.53)
#         beta = ngg / dgg
        
#         w_old = w
#         g = h
#         # Mise à jour de la direction de descente
#         p = -g + beta * p
        
#         print(f"Epoque {epoque} and loss is: {new_loss}")
#         epoque +=1

In [78]:
w = np.random.normal(10, 2, 11)

In [79]:
w

array([10.11551837,  9.04289943,  9.53303687, 11.45298556,  5.0926688 ,
        9.89889583, 11.81427155, 10.875974  , 12.44586153,  7.32282205,
        6.76311454])

In [80]:
opt.conjugate_gradient(X=X, y=y, w=w, 
                   cost_func=lg.logistic_surrogate_loss, 
                   grd_func=lg.gradient_logistic_surrogate_loss, epsilon=0.01)

Epoque 0 and loss is: 0.25


In [42]:
# def conjugate_gradient(X, y, w, cost_func, grd_func, eps=0.1):
#     # Random initialization of weights
#     m, d = X.shape
#     w = np.random.random(d)
    
#     # Initializing losses
#     L = cost_func(X, y, w)
#     Lold = L + 2*eps
#     g, g0 = grd_func(X, y, w)
#     p, p0 = -g, -g0
#     t = 0
# #     line_search(X, y, w, cost_func, g, w, Lold, g, g0, p, p0, L)
#     while np.abs(Lold - L) > (np.abs(Lold) * eps):
        
# #        
#         line_search(X, y, w, cost_func, g, w, Lold, g, g0, p, p0, L)
#         # line search is supposed to modify the values of w ou initialiser.
#         # creating new gradient vector h
# #         h = np.gradient(cost_func(X, y, w))
#         h, h0 = grd_func(X, y, w)
#         dgg = np.linalg.norm(g)
#         ngg = np.linalg.norm(h)
#         beta = dgg / ngg # Formule de Fletcher-Reeves (Eq 2.53)
#         wold = w
#         g = h
#         h0 = g0
#         p = -g + beta*p # MAJ de la direction de descente (Eq 2.46)

In [44]:
ps**2

array([6.86723301e-02, 1.30092141e+00, 2.81690443e-01, 1.75550138e+01,
       1.69625433e+01, 7.15832083e+00, 2.58673358e+01, 9.38929142e-04,
       4.32863644e-01, 2.06890272e+01, 7.30251962e-01, 4.31174967e+01,
       9.20162267e+00, 8.86395687e+00, 1.82140622e+01, 5.96392065e+00,
       2.80494125e-01, 6.17969920e+00, 3.38321492e+00, 1.76053634e+01,
       4.13623696e+00, 1.28287528e+01, 4.06665988e+01, 7.05002805e+00,
       3.47819821e+01, 1.81896152e+00, 8.20099310e+00, 1.03261306e+00,
       5.97677111e+00, 1.15142922e+01, 1.06222105e+01, 1.84301508e+00,
       5.10818820e+00, 6.14204552e+00, 7.14195460e+00, 8.13264511e+00,
       6.27167749e-01, 1.66181666e+01, 7.89596888e+00, 1.61439182e+01,
       5.70520405e-01, 8.59571354e+00, 9.11347900e-01, 2.20958319e+00,
       3.90684813e+00, 1.63583084e+01, 1.87595811e-02, 4.98350851e+00,
       1.29920469e-01, 6.74983224e+00, 8.38063082e+00, 1.87636957e+01,
       1.91543772e+01, 2.99089939e+00, 1.06003174e+01, 2.23740490e+00,
      

# Test des algos sur un table

In [82]:
import pandas as pd
breast_cancer = pd.read_csv('../../DB/breast-cancer-wisconsin.data', sep=',', header=None)
breast_cancer.columns = ['col' + str(i) for i in range(len(breast_cancer.columns))]

In [83]:
breast_cancer.drop('col6', inplace=True, axis=1)

In [ ]:
X = breast_cancer.iloc[:, 1:-1]
y = breast_cancer.iloc[:, -1]
y[y==2] = -1
y[y==4] = 1

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    test_size=.4, shuffle=True)

In [93]:
opt.conjugate_gradient(X=X_train, y=y_train, w=w, 
                   cost_func=lg.logistic_surrogate_loss, 
                   grd_func=lg.gradient_logistic_surrogate_loss, epsilon=0.1)

Epoque 0 and loss is: 0.5351282124498348
Epoque 1 and loss is: 0.26666666666666666
Epoque 2 and loss is: 0.26666666666666666


In [87]:
w

array([10.11551837,  9.04289943,  9.53303687, 11.45298556,  5.0926688 ,
        9.89889583, 11.81427155, 10.875974  , 12.44586153,  7.32282205,
        6.76311454])